In [1]:
#from google.colab import drive
#drive.mount("/content/drive", force_remount=True)

In [2]:
#%cd /content/drive/MyDrive/Masters_Project/Datasets/OASIS3/

In [3]:
!ls /tmp/data

OAS30001_MR_d0129  OAS30226_MR_d0183  OAS30463_MR_d2737  OAS30737_MR_d0035
OAS30001_MR_d0757  OAS30227_MR_d0000  OAS30464_MR_d0077  OAS30737_MR_d1137
OAS30001_MR_d2430  OAS30228_MR_d0090  OAS30464_MR_d0951  OAS30739_MR_d0051
OAS30001_MR_d3132  OAS30229_MR_d0101  OAS30464_MR_d2445  OAS30740_MR_d0069
OAS30002_MR_d0371  OAS30230_MR_d3855  OAS30464_MR_d2848  OAS30741_MR_d3184
OAS30002_MR_d0653  OAS30231_MR_d1469  OAS30465_MR_d0152  OAS30742_MR_d0058
OAS30002_MR_d2340  OAS30231_MR_d2510  OAS30466_MR_d0058  OAS30742_MR_d3401
OAS30002_MR_d2345  OAS30231_MR_d3617  OAS30466_MR_d0671  OAS30742_MR_d3619
OAS30003_MR_d0558  OAS30231_MR_d3630  OAS30467_MR_d0092  OAS30742_MR_d4551
OAS30003_MR_d1631  OAS30232_MR_d0120  OAS30468_MR_d0069  OAS30743_MR_d0051
OAS30003_MR_d2669  OAS30232_MR_d0695  OAS30469_MR_d0112  OAS30743_MR_d0645
OAS30003_MR_d2682  OAS30232_MR_d2324  OAS30470_MR_d1023  OAS30743_MR_d2309
OAS30003_MR_d3320  OAS30233_MR_d3867  OAS30470_MR_d1846  OAS30744_MR_d0063
OAS30003_MR_d3731  OAS302

In [4]:
!pip install -r ./ganomaly/requirements.txt

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
!pip install mkl-fft

     |████████████████████████████████| 919 kB 1.6 MB/s eta 0:00:01
  Attempting uninstall: intel-openmp
    Found existing installation: intel-openmp 2021.1.2
    Uninstalling intel-openmp-2021.1.2:
      Successfully uninstalled intel-openmp-2021.1.2
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

mkl 2021.1.1 requires intel-openmp==2021.*, but you'll have intel-openmp 2020.0.133 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
#!pip install Pillow==6.2.2

# imports

In [7]:
import os
#import os.path
#import cv2
import glob
import torch
import random
import numpy as np
import pickle
from typing import Any, Callable, Optional, Tuple

from PIL import Image

from sklearn.model_selection import train_test_split

import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import check_integrity, download_and_extract_archive

In [8]:
print(torch.__version__)

1.2.0


In [9]:
import torchvision
torchvision.__version__

'0.4.0'

# Create Oasis3 data class

In [10]:
splits = ['train', 'test']
drop_last_batch = {'train': True, 'test': False}
shuffle = {'train': True, 'test': True}

transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.ToTensor(),
        #transforms.Normalize([0.5], [0.5])# https://github.com/yunjey/pytorch-tutorial/issues/161
        transforms.Normalize((0.1307,), (0.3081,))
        
    ]
)

In [11]:
train_image_number = 140 # Image slice from MRI scans to use for training data
#root_dir = '/content/drive/MyDrive/Masters_Project/Datasets/OASIS3/'
root_dir = '/tmp/data/'
base_folder = '/pyt_oasis3'
non_ad_jpg_folder = 't1w_non_ad_jpgs/' + str(train_image_number) + '/'
ad_jpg_folder = 't1w_ad_jpgs/' + str(train_image_number) + '/'

In [12]:
class Oasis3(VisionDataset):
    """
    NOTE: The constructor automatically shuffles and samples ad and non-ad images at a 80:20 ratio
    ad_jpg_folder: system path with ad jpg images
    non_ad_jpg_folder: system path with non ad images
    """
    def __init__(self,
            root: str,
            ad_jpg_folder = None,
            non_ad_jpg_folder = None,
            train: bool = True,
            transform: Optional[Callable] = None,
            target_transform: Optional[Callable] = None) -> None:

        self.ad_jpg_folder = os.path.join(root, ad_jpg_folder)
        self.non_ad_jpg_folder = os.path.join(root, non_ad_jpg_folder)
            
        print("="*5 + " Loading Data " + "="*5)
        print(f"ad_jpg_folder: {self.ad_jpg_folder}")
        print(f"non_ad_jpg_folder: {self.non_ad_jpg_folder}")
            
        super(Oasis3, self).__init__(root, transform=transform, target_transform=target_transform)
        self.train = train 
        
        self.non_ad_data: Any = []
        self.non_ad_targets = []

        self.ad_data: Any = []
        self.ad_targets = []
        
        # Read data and label into a numpy array
        self.non_ad_data, self.non_ad_targets = self.read_data(self.ad_jpg_folder+'**.jpg', normal=True)
        self.ad_data, self.ad_targets = self.read_data(self.non_ad_jpg_folder+'**.jpg', normal=False)
        
        # shuffle the datasets and bucket them as test and train
        (self.non_ad_data, self.non_ad_targets) = self.shuffle_associated_arrays(self.non_ad_data, self.non_ad_targets)
        (self.ad_data, self.ad_targets) = self.shuffle_associated_arrays(self.ad_data, self.ad_targets)

        # concat the lists and random shuffle
        all_data = np.concatenate((self.non_ad_data, self.ad_data), axis=0)
        all_targets = self.non_ad_targets + self.ad_targets

        random.Random(1).shuffle(all_data)
        random.Random(1).shuffle(all_targets)

        # Use scikit learn to split complete data into test and train
        self.train_data, self.test_data, self.train_targets, self.test_targets =\
         train_test_split(all_data, all_targets, test_size=0.1, random_state=42)

        # Set test or train data based on input
        if self.train:
          self.data = self.train_data
          self.targets = self.train_targets
        else:
          self.data = self.test_data
          self.targets = self.test_targets
            
    def read_data(self, path, normal=True):
        data: Any = []
        labels = []
        
        for file_path in glob.glob(path):
            with open (file_path, 'rb') as f:
                # image needs to be a PIL image
                img = Image.open(f)
                # Resize all images 176, 256, 3 -> 256, 256, 0
                dsize = (256, 256)
                resized = img.resize(dsize)
                data.append(resized)
                labels.append(1 if normal else 0) 
        data = np.vstack(data).reshape(-1, 256, 256)
        data = data.transpose((0, 1, 2))  # convert to HWC
        return (data, labels)

    def shuffle_associated_arrays(self, data, targets):
        c = list(zip(data, targets))
        random.shuffle(c)
        data, targets = zip(*c)
        return (np.array(data), list(targets))

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        img, target = self.data[index], self.targets[index]
        
        img = Image.fromarray(img)
        
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return (img, target)

In [13]:
# Issue is i need to make the data transformable inside __getitem__
#a=Oasis3(base_folder, False, transform=transform)

dataset = {}
dataset['train'] = Oasis3(root_dir, ad_jpg_folder, non_ad_jpg_folder, train=True, transform=transform)
dataset['test'] = Oasis3(root_dir, ad_jpg_folder, non_ad_jpg_folder, train=False, transform=transform)

===== Loading Data =====
ad_jpg_folder: /tmp/data/t1w_ad_jpgs/140/
non_ad_jpg_folder: /tmp/data/t1w_non_ad_jpgs/140/
===== Loading Data =====
ad_jpg_folder: /tmp/data/t1w_ad_jpgs/140/
non_ad_jpg_folder: /tmp/data/t1w_non_ad_jpgs/140/


In [14]:
# The class should be index into which invokes the __getitem__ call
print(dataset['train'][0][0].shape)
print(dataset['train'][0][0])
print(dataset['train'][0][1])

torch.Size([1, 256, 256])
tensor([[[-0.3988, -0.3860, -0.3988,  ..., -0.3860, -0.3860, -0.4242],
         [-0.3988, -0.3606, -0.3733,  ..., -0.3860, -0.3860, -0.4242],
         [-0.3988, -0.3478, -0.3224,  ..., -0.3988, -0.3988, -0.4242],
         ...,
         [-0.2969, -0.3224, -0.3733,  ..., -0.4242, -0.3860, -0.4242],
         [-0.3478, -0.3606, -0.3988,  ..., -0.4242, -0.3860, -0.4242],
         [-0.4242, -0.3860, -0.3988,  ..., -0.4242, -0.3733, -0.4242]]])
1


In [15]:
def get_oasis_anomaly_dataset(trn_img, trn_lbl, tst_img, tst_lbl, abn_cls_idx=1, manualseed=-1):
    """[summary]
    Arguments:
        trn_img {np.array} -- Training images
        trn_lbl {np.array} -- Training labels
        tst_img {np.array} -- Test     images
        tst_lbl {np.array} -- Test     labels
    Keyword Arguments:
        abn_cls_idx {int} -- Anomalous class index (default: {1})
    Returns:
        [np.array] -- New training-test images and labels.
    """
    # Convert train-test labels into numpy array.
    trn_lbl = np.array(trn_lbl)
    tst_lbl = np.array(tst_lbl)

    # --
    # Find idx, img, lbl for abnormal and normal on org dataset.
    nrm_trn_idx = np.where(trn_lbl != abn_cls_idx)[0]
    abn_trn_idx = np.where(trn_lbl == abn_cls_idx)[0]

    nrm_trn_img = trn_img[nrm_trn_idx]    # Normal training images
    abn_trn_img = trn_img[abn_trn_idx]    # Abnormal training images
    nrm_trn_lbl = trn_lbl[nrm_trn_idx]    # Normal training labels
    abn_trn_lbl = trn_lbl[abn_trn_idx]    # Abnormal training labels.

    # nrm_trn_img = [trn_img[i] for i in nrm_trn_idx]
    # abn_trn_img = [trn_img[i] for i in abn_trn_idx]
    # nrm_trn_lbl = [trn_lbl[i] for i in nrm_trn_idx]
    # abn_trn_img = [trn_lbl[i] for i in abn_trn_idx]  

    nrm_tst_idx = list(np.where(tst_lbl != abn_cls_idx))[0]
    abn_tst_idx = list(np.where(tst_lbl == abn_cls_idx))[0]

    nrm_tst_img = tst_img[nrm_tst_idx]    # Normal training images
    abn_tst_img = tst_img[abn_tst_idx]    # Abnormal training images.
    nrm_tst_lbl = tst_lbl[nrm_tst_idx]    # Normal training labels
    abn_tst_lbl = tst_lbl[abn_tst_idx]    # Abnormal training labels.

    # nrm_tst_img = [tst_img[i] for i in nrm_tst_idx]
    # abn_tst_img = [tst_img[i] for i in abn_tst_idx]
    # nrm_tst_lbl = [trn_lbl[i] for i in nrm_tst_idx]
    # abn_tst_lbl = [trn_lbl[i] for i in abn_tst_idx]  

    # --
    # Assign labels to normal (0) and abnormals (1)
    nrm_trn_lbl[:] = 0
    nrm_tst_lbl[:] = 0
    abn_trn_lbl[:] = 1
    abn_tst_lbl[:] = 1

    # --
    if manualseed != -1:
        # Random seed.
        # Concatenate the original train and test sets.
        nrm_img = np.concatenate((nrm_trn_img, nrm_tst_img), axis=0)
        nrm_lbl = np.concatenate((nrm_trn_lbl, nrm_tst_lbl), axis=0)
        abn_img = np.concatenate((abn_trn_img, abn_tst_img), axis=0)
        abn_lbl = np.concatenate((abn_trn_lbl, abn_tst_lbl), axis=0)

        # Split the normal data into the new train and tests.
        idx = np.arange(len(nrm_lbl))
        np.random.seed(manualseed)
        np.random.shuffle(idx)

        nrm_trn_len = int(len(idx) * 0.80)
        nrm_trn_idx = idx[:nrm_trn_len]
        nrm_tst_idx = idx[nrm_trn_len:]

        nrm_trn_img = nrm_img[nrm_trn_idx]
        nrm_trn_lbl = nrm_lbl[nrm_trn_idx]
        nrm_tst_img = nrm_img[nrm_tst_idx]
        nrm_tst_lbl = nrm_lbl[nrm_tst_idx]

    # Create new anomaly dataset based on the following data structure:
    # - anomaly dataset
    #   . -> train
    #        . -> normal
    #   . -> test
    #        . -> normal
    #        . -> abnormal
    new_trn_img = np.copy(nrm_trn_img)
    new_trn_lbl = np.copy(nrm_trn_lbl)
    new_tst_img = np.concatenate((nrm_tst_img, abn_trn_img, abn_tst_img), axis=0)
    new_tst_lbl = np.concatenate((nrm_tst_lbl, abn_trn_lbl, abn_tst_lbl), axis=0)

    return new_trn_img, new_trn_lbl, new_tst_img, new_tst_lbl

In [16]:
dataset['train'].data, dataset['train'].targets, \
dataset['test'].data, dataset['test'].targets = get_oasis_anomaly_dataset(
    trn_img=dataset['train'].data,
    trn_lbl=dataset['train'].targets,
    tst_img=dataset['test'].data,
    tst_lbl=dataset['test'].targets,
    abn_cls_idx=1
)

In [17]:
!pwd

/notebook/Masters_Project/wip_colabs


In [18]:
!ls

1-oasis3-t1w-cdr-split.ipynb	3-Save_ad-jpg-slices.ipynb  Test-Ganomaly.ipynb
2-Save_non-ad-jpg-slices.ipynb	4-load_oasis3_data.ipynb    ganomaly


In [19]:
%env PYTHONPATH="$/env/python:/notebook/Masters_Project/wip_colabs/ganomaly:/notebook/Masters_Project/wip_colabs/ganomaly/lib"
!echo $PYTHONPATH

env: PYTHONPATH="$/env/python:/notebook/Masters_Project/wip_colabs/ganomaly:/notebook/Masters_Project/wip_colabs/ganomaly/lib"
"$/env/python:/notebook/Masters_Project/wip_colabs/ganomaly:/notebook/Masters_Project/wip_colabs/ganomaly/lib"


In [20]:
from __future__ import print_function

from ganomaly.options import Options
from ganomaly.lib.data import load_data

In [22]:
#from ganomaly.lib.model import Ganomaly

In [23]:
%cd ganomaly

/notebook/Masters_Project/wip_colabs/ganomaly


In [24]:
!python train.py --dataset oasis3 --isize 256 --nc 1 --niter 15 --abnormal_class 1

Traceback (most recent call last):
  File "train.py", line 43, in <module>
    train()
  File "train.py", line 34, in train
    dataloader = load_data(opt)
  File "/notebook/Masters_Project/wip_colabs/ganomaly/lib/data.py", line 166, in load_data
    dataset = {x: ImageFolder(os.path.join(opt.dataroot, x), transform) for x in splits}
  File "/notebook/Masters_Project/wip_colabs/ganomaly/lib/data.py", line 166, in <dictcomp>
    dataset = {x: ImageFolder(os.path.join(opt.dataroot, x), transform) for x in splits}
  File "/usr/local/lib/python3.6/dist-packages/torchvision/datasets/folder.py", line 209, in __init__
    is_valid_file=is_valid_file)
  File "/usr/local/lib/python3.6/dist-packages/torchvision/datasets/folder.py", line 93, in __init__
    classes, class_to_idx = self._find_classes(self.root)
  File "/usr/local/lib/python3.6/dist-packages/torchvision/datasets/folder.py", line 122, in _find_classes
    classes = [d.name for d in os.scandir(dir) if d.is_dir()]
FileNotFoundError: [

In [25]:
!pwd

/notebook/Masters_Project/wip_colabs/ganomaly


ls: cannot access './ganomaly': No such file or directory


In [ ]:
"""

root@6f3d4f61bc71:/notebook/Masters_Project/ganomaly# python train.py --dataset oasis3 --isize 256 --nc 1 --niter 15 --abnormal_class 1                                                                                                                      
Getting data via dataloader
Loading Oasis3 dataset
===== Loading Data =====
ad_jpg_folder: /tmp/data/t1w_ad_jpgs/140/
non_ad_jpg_folder: /tmp/data/t1w_non_ad_jpgs/140/
===== Loading Data =====
ad_jpg_folder: /tmp/data/t1w_ad_jpgs/140/
non_ad_jpg_folder: /tmp/data/t1w_non_ad_jpgs/140/
Creating oasis3 dataloader
Return oasis3 dataloader
Loading Ganomaly model classes
Training Model
>> Training model Ganomaly.
  0%|                                                                                                                                                                                                                                 | 0/15 [00:00<?, ?it/s]Traceback (most recent call last):
  File "train.py", line 46, in <module>
    train()
  File "train.py", line 43, in train
    model.train()
  File "/notebook/Masters_Project/ganomaly/lib/model.py", line 166, in train
    self.train_one_epoch()
  File "/notebook/Masters_Project/ganomaly/lib/model.py", line 129, in train_one_epoch
    for data in tqdm(self.dataloader['train'], leave=False, total=len(self.dataloader['train'])):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 1032, in __iter__
    for obj in iterable:
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 819, in __next__
    return self._process_data(data)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 846, in _process_data
    data.reraise()
  File "/usr/local/lib/python3.6/dist-packages/torch/_utils.py", line 369, in reraise
    raise self.exc_type(msg)
RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/notebook/Masters_Project/ganomaly/lib/Oasis3.py", line 105, in __getitem__
    img = self.transform(img)
  File "/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py", line 61, in __call__
    img = t(img)
  File "/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py", line 166, in __call__
    return F.normalize(tensor, self.mean, self.std, self.inplace)
  File "/usr/local/lib/python3.6/dist-packages/torchvision/transforms/functional.py", line 217, in normalize
    tensor.sub_(mean[:, None, None]).div_(std[:, None, None])
RuntimeError: output with shape [1, 256, 256] doesn't match the broadcast shape [3, 256, 256]


"""